# Tools

This notebook contains all the tools that will be used by the agent.

In [1]:
import requests
import sqlite3
import pandas as pd

from smolagents import tool, Tool

from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun

## Creating simple tool

A Python function must be annotated with `@tool`. It should also have a docstring describing what does the function do, what does it return and the description of its parameters.

### City to location 

The following function look up at latitude and longitude of a city.

In [2]:
# TODO: Load CSV file containing latitude, longitude and altitude of cities
# https://github.com/bahar/WorldCityLocations/tree/master
df = pd.read_csv('data/cities_latlng.csv', sep=';')
df.head()

,1,Afghanistan,Kabul,34.5166667,69.1833344,1808
0,2,Afghanistan,Kandahar,31.610000,65.699997,1015
1,3,Afghanistan,Mazar-e Sharif,36.706944,67.112221,369
2,4,Afghanistan,Herat,34.340000,62.189999,927
3,5,Afghanistan,Jalalabad,34.420000,70.449997,573
4,6,Afghanistan,Konduz,36.720000,68.860001,394


In [3]:
df.columns = ['id', 'country', 'city', 'latitude', 'longitude', 'altitude']
df.drop('country', axis=1, inplace=True)

In [4]:
df.head()

,id,city,latitude,longitude,altitude
0,2,Kandahar,31.610000,65.699997,1015
1,3,Mazar-e Sharif,36.706944,67.112221,369
2,4,Herat,34.340000,62.189999,927
3,5,Jalalabad,34.420000,70.449997,573
4,6,Konduz,36.720000,68.860001,394


In [ ]:
# TODO: Explore the loaded dataframe


In [10]:
# TODO: Add tool description

@tool
def get_latlng(city: str) -> any:
   """
   Get the latitude, longitude and altitude of a given city, county, state, etc.

   Args:
      city: the name of the city that you want to get the latitude, longitude and altitude

   Returns:
      any: a dictionary with the following keys: city, latitude, longitude and altitude

   Example:
      result = get_latlng('tokyo')
   """
   r = df.query(f"city.str.lower() == '{city.lower()}'")
   return { 'city': city, 'latitude': r.iloc[0]['latitude'], 'longitude': r.iloc[0]['longitude'], 'altitude': r.iloc[0]['altitude'] }

In [11]:
# TODO: Test get_latlng method
# case insensitive search
get_latlng('tokyo')

{'city': 'tokyo',
 'latitude': np.float64(35.6895266),
 'longitude': np.float64(139.6916809),
 'altitude': np.int64(40)}

### Temperature at latitude and longitude

The following function lookup the weather at the given latitude and longtude.

In [14]:
# TODO: Add tool description

@tool
def get_temperature(latitude: float, longitude: float) -> any:
   """ 
   Return the temperature of a location given by its latitude and logitude

   Args:
      latitude: location's latitude
      longitude: location's longitude

   Returns:
      any: a dictionary with the following keys: temperature_unit in celsius, temperature 

   Example:
      result = get_temperature(35.6895266, 139.6916809)
   """
   url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m"
   resp = requests.get(url)
   j = resp.json()
   if resp.status_code >= 400:
      raise Exception(j['reason'])
   temperature = j['current']['temperature_2m']
   units = j['current_units']['temperature_2m']
   return { "temperature_unit": units, "temperature": temperature }

In [13]:
# TODO: Test get_temperature method
get_temperature(35.6895266, 139.6916809)

{'temperature_unit': '°C', 'temperature': 30.5}

In [ ]:
# TODO: Create a web search tool

def web_search(query: str) -> any:

   search = DuckDuckGoSearchRun()
   return search.invoke(query)

In [ ]:
# TODO: Test the web search


### Query relational database

The following function queries a relational database (SQLite) view called `album_track`. The table's schema is as follows:
| Field name  | Type          |
|-------------|---------------| 
| AlbumId     | integer       |
| Title       | nvarchar(160) |
| track_name  | nvarchar(200) |
| artist_name | nvarchar(120) |
| duration    | integer       |
| composer    | nvarchar(220) |


In [19]:
# TODO: Add tool description
@tool
def query_album_track(query: str) -> str:
   """ 
   Perform SQL queries on the album_track table. Returns the result as an array of records.
   The table name is album_track and has the following columns:
      AlbumId: integer
      Title: nvachar(160)
      track_name: nvachar(200)
      artist_name: nvachar(120)
      duration: integer
      composer nvachar(220)
   The duration column is in milliseconds

   Args:
      query: a valid SQLite SQL query

   Returns:
      any: list of tuple. Each element corresponds to a record

   Example:
      results = query_album_track('select * from album_track)
   """
   database = "data/chinook_sqlite.sqlite" 
   conn = sqlite3.connect(database)
   try:
      cursor = conn.cursor() 
      rows = cursor.execute(query)
      return rows.fetchall()
   finally:
      conn.close()

In [20]:
# TODO: Test the query_album_track function
query_album_track('select * from album_track limit 3')

[(1,
  'For Those About To Rock We Salute You',
  'For Those About To Rock (We Salute You)',
  'AC/DC',
  343719,
  'Angus Young, Malcolm Young, Brian Johnson'),
 (2, 'Balls to the Wall', 'Balls to the Wall', 'Accept', 342562, None),
 (3,
  'Restless and Wild',
  'Fast As a Shark',
  'Accept',
  230619,
  'F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman')]

### Tools with states

The following isn an example of a more complex tool that requires initialisation

In [31]:
class SQLiteTool(Tool):
   # required metadata
   name = "chinook_sqlite"

   description = """ 
   Perform SQL queries on the album_track table. Returns the result as an array of records.
   The table name is album_track and has the following columns:
      AlbumId: integer
      Title: nvachar(160)
      track_name: nvachar(200)
      artist_name: nvachar(120)
      duration: integer
      composer nvachar(220)
   The duration column is in milliseconds
   """

   inputs = {
      "query": {
         "type": "string",
         "description": "a valid SQLite SQL query"
      }   
   }

   output_type = "any"

   def __init__(self, db_file):
      self.db_file = db_file 
      self.conn = sqlite3.connect(self.db_file)
      self.setup()

   def forward(self, query: str) -> str:      
      try:
         cursor = self.conn.cursor() 
         rows = cursor.execute(query)
         return rows.fetchall()
      finally:
         pass  

In [32]:
sql = SQLiteTool("data/chinook_sqlite.sqlite")

In [33]:
sql.forward('select count(*) from album_track')

[(3503,)]